# Download and Format Lawmatics Invoices

This Notebook downloads all matters from Lawmatics, formats them into a tabular format, applies data types, then loads them into a BigQuery datastore table for further processing.

## Install Requirements

In [9]:
!pip install google.cloud google-cloud-secret-manager

## Retrieve Lawmatics API Token

In [10]:
#!/usr/bin/env python3
from google.cloud import secretmanager
from hashlib import sha1



def access_secret(project_id, secret_id, version_id="latest"):
    """
    Access a secret from Google Cloud Secret Manager

    Args:
        project_id: Your Google Cloud project ID
        secret_id: The ID of the secret
        version_id: The version of the secret (defaults to "latest")

    Returns:
        The secret value as a string
    """
    # Create the Secret Manager client
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version
    response = client.access_secret_version(request={"name": name})

    # Return the decoded secret
    return response.payload.data.decode("UTF-8")

# Example usage
if __name__ == "__main__":
    project_id = '566451752897'
    secret_id = 'LM_API_TOKEN'

    try:
        secret_value = access_secret(project_id, secret_id)
        secret_value_sha1 = sha1(secret_value.encode())
        secret_value_digest = secret_value_sha1.hexdigest()
        print(f"Secret retrieved successfully: (sha1) {secret_value_digest}")
    except Exception as e:
        print(f"Error accessing secret: {e}")

Secret retrieved successfully: (sha1) 9793ec7db1c271bf0898ec6a4423c80a863799e6


## Get API JSON Data
Retrieve paginated data from a REST API using a non-expiring OAuth2 access token and transform it into a tabular format for analysis.

### Set up api request

#### Subtask:
Define the API endpoint, headers (including the access token), and any initial parameters for the first page of data.


In [11]:
api_endpoint = 'https://api.lawmatics.com/v1/invoices'
headers = {
    'Authorization': f'Bearer {secret_value}'
}
params = {
    'fields': 'all',
    'page': 1,
    'per_page': 100 # Or any other appropriate value based on API documentation
}

### Implement pagination logic

#### Subtask:
Handle the API's pagination, making requests for subsequent pages until all data is retrieved.


In [12]:
import requests
import time

all_data = []
page_num = 1
max_retries = 3
retry_delay = 5 # seconds

# api_endpoint is already defined in a previous cell
# headers and params are already defined in a previous cell

# Remove the Cookie header as it's not necessary according to API documentation
if 'Cookie' in headers:
    del headers['Cookie']


while True:
    params['page'] = page_num
    retries = 0
    while retries < max_retries:
        try:
            # Allow redirects by default (requests handles --location)
            response = requests.get(api_endpoint, headers=headers, params=params)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            page_data = response.json()

            # Check the structure of page_data based on the likely API response.
            # Assuming the API returns data in a key like 'data' and pagination info in 'meta'
            # If the API response structure is different, this part needs to be adjusted.
            if 'data' in page_data and isinstance(page_data['data'], list):
                current_page_items = page_data['data']
                if not current_page_items:  # Check if the 'data' list is empty
                    print("Empty data list received, assuming end of data.")
                    break # No more data

                all_data.extend(current_page_items)
                page_num += 1
                time.sleep(1) # Add a small delay to avoid overwhelming the API
                break # Break out of retry loop on success
            else:
                print("API response does not contain expected 'data' key with a list.")
                # Depending on the actual API response, you might want to
                # inspect page_data here to understand the structure.
                print(f"Response content: {page_data}")
                break # Exit if response structure is unexpected


        except requests.exceptions.RequestException as e:
            retries += 1
            print(f"Error during API request (Attempt {retries}/{max_retries}): {e}")
            if retries < max_retries:
                time.sleep(retry_delay)
            else:
                print("Max retries reached. Exiting.")
                break # Exit retry loop after max retries

    if retries == max_retries or ('data' in page_data and not page_data['data']):
        break # Exit main loop if max retries reached or no data received in the last successful attempt

print(f"Retrieved {len(all_data)} total records.")

Empty data list received, assuming end of data.
Retrieved 284 total records.


## Process and convert data to tabular format

Convert the retrieved data from the API response (likely JSON) into a tabular format, such as a pandas DataFrame.


In [13]:
import pandas as pd

df = pd.DataFrame(all_data)
display(df.head())
display(df.info())

,id,type,attributes,relationships
0,359893,invoice,"{'number': 900686, 'status': 'paid', 'target_a...","{'invoice_type': {'data': None}, 'prospect': {..."
1,359180,invoice,"{'number': 900685, 'status': 'paid', 'target_a...","{'invoice_type': {'data': None}, 'prospect': {..."
2,357564,invoice,"{'number': 900684, 'status': 'paid', 'target_a...","{'invoice_type': {'data': None}, 'prospect': {..."
3,355890,invoice,"{'number': 900682, 'status': 'paid', 'target_a...","{'invoice_type': {'data': None}, 'prospect': {..."
4,354224,invoice,"{'number': 900681, 'status': 'sent', 'target_a...","{'invoice_type': {'data': None}, 'prospect': {..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             284 non-null    object
 1   type           284 non-null    object
 2   attributes     284 non-null    object
 3   relationships  284 non-null    object
dtypes: object(4)
memory usage: 9.0+ KB


None

### Extract nested data from columns

Extract relevant information from the 'attributes' and 'relationships' columns and create new columns in the DataFrame.

In [14]:
import pandas as pd
import re

# Extract data from the 'attributes' column if it exists
attributes_df = pd.DataFrame() # Initialize as empty
if 'attributes' in df.columns:
    attributes_df = df['attributes'].apply(pd.Series)
    df = df.drop(columns=['attributes'])


# Modified function to extract all relevant data from the 'relationships' column
# This function will now return a dictionary for each row
def extract_relationship_data_generic(relationship):
    extracted = {}
    if isinstance(relationship, dict):
        for key, value in relationship.items():
            if isinstance(value, dict) and 'data' in value:
                data_item = value['data']
                if isinstance(data_item, dict):
                    # For single related items, extract id and type
                    extracted[f"{key}_id"] = data_item.get('id')
                    extracted[f"{key}_type"] = data_item.get('type')
                elif isinstance(data_item, list):
                    # For lists of related items, join IDs and types
                    ids = [item.get('id') for item in data_item if isinstance(item, dict)]
                    types = [item.get('type') for item in data_item if isinstance(item, dict)]
                    # Use filter(None, ...) to remove None values before joining
                    extracted[f"{key}_ids"] = ','.join(filter(None, ids)) if ids else None
                    extracted[f"{key}_types"] = ','.join(filter(None, types)) if types else None
    return extracted


relationships_df_expanded = pd.DataFrame() # Initialize as empty
if 'relationships' in df.columns:
    # Apply the generic extraction function
    relationships_extracted_series = df['relationships'].apply(extract_relationship_data_generic)

    # Convert the Series of dictionaries into a DataFrame
    # This will create columns dynamically based on the keys present in the dictionaries
    relationships_df_expanded = pd.DataFrame(relationships_extracted_series.tolist())

    # Drop the original 'relationships' column
    df = df.drop(columns=['relationships'])


# Process 'custom_fields' column: Rename to 'custom_field_values' and keep as structured data
if 'custom_fields' in df.columns:
    df = df.rename(columns={'custom_fields': 'custom_field_values'})


# Final concatenation of all parts
# Ensure all parts have their index reset for correct concatenation.
all_parts = []

# Add the base dataframe (containing 'id', 'type', and other top-level columns after 'attributes' and 'relationships' drops)
# The 'custom_field_values' column (if present) is now part of this df
all_parts.append(df.reset_index(drop=True))

# Add the attributes dataframe if it's not empty
if not attributes_df.empty:
    all_parts.append(attributes_df.reset_index(drop=True))

# Add the relationships dataframe if it's not empty:
if not relationships_df_expanded.empty:
    all_parts.append(relationships_df_expanded.reset_index(drop=True))

# Note: custom_fields_df_expanded is no longer created or appended here,
# as the custom_field_values column is kept as structured data in the main df.

# Concatenate all parts along columns, filling missing values with NaN.
df = pd.concat(all_parts, axis=1, join='outer')

# Convert common date fields to datetime objects
date_columns_to_convert = [
    'created_at', 'updated_at', 'closed_at', 'last_activity_at',
    'intake_form_completed_at', 'first_contact_at', 'referral_date',
    'follow_up_date', 'opened_at', 'sent_at', 'due_at', 'start_date', 'end_date'
]

for col in date_columns_to_convert:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)

# Convert NaN values to None for all columns
df = df.where(pd.notna(df), None)

# Function to clean column names for BigQuery compatibility
def clean_column_name(col_name):
    # Replace spaces with underscores
    col_name = col_name.replace(' ', '_')
    # Remove characters that are not alphanumeric or underscores
    col_name = re.sub(r'[^0-9A-Za-z_]', '', col_name)
    # Convert to lowercase
    col_name = col_name.lower()
    # Ensure it doesn't start with a number (BigQuery requirement)
    if col_name and col_name[0].isdigit():
        col_name = '_' + col_name
    # Truncate to BigQuery's max length (300 characters) if necessary
    col_name = col_name[:300]
    return col_name

# Apply column name cleaning to all columns in the DataFrame
df.columns = [clean_column_name(col) for col in df.columns]

# Display the first few rows of the updated DataFrame to verify the changes
display(df.head())
display(df.info())

,id,type,number,status,target_account_type,payment_terms,include_balance_forward,enable_online_payment,include_statement_of_accounts,invoiced_at,...,invoice_adjustments_ids,invoice_adjustments_types,transactions_ids,transactions_types,forwarded_from_ids,forwarded_from_types,invoice_type_id,invoice_type_type,forwarded_to_id,forwarded_to_type
0,359893,invoice,900686,paid,operating,due_on_receipt,False,True,False,2026-01-09,...,None,None,571294,transaction,None,None,None,None,None,None
1,359180,invoice,900685,paid,operating,due_on_receipt,False,True,False,2026-01-09,...,None,None,"570534,570535","transaction,transaction",None,None,None,None,None,None
2,357564,invoice,900684,paid,operating,due_on_receipt,False,True,False,2026-01-07,...,None,None,"568751,568752","transaction,transaction",None,None,None,None,None,None
3,355890,invoice,900682,paid,operating,due_on_receipt,False,False,False,2026-01-05,...,None,None,"563222,563223","transaction,transaction",None,None,None,None,None,None
4,354224,invoice,900681,sent,operating,net30,False,True,False,2025-12-31,...,None,None,None,None,None,None,None,None,None,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 40 columns):
 #   Column                         Non-Null Count  Dtype              
---  ------                         --------------  -----              
 0   id                             284 non-null    object             
 1   type                           284 non-null    object             
 2   number                         284 non-null    int64              
 3   status                         284 non-null    object             
 4   target_account_type            284 non-null    object             
 5   payment_terms                  284 non-null    object             
 6   include_balance_forward        284 non-null    bool               
 7   enable_online_payment          284 non-null    bool               
 8   include_statement_of_accounts  284 non-null    bool               
 9   invoiced_at                    284 non-null    object             
 10  due_at                    

None

In [15]:
with pd.option_context('display.max_rows', None):
    print(df.dtypes)

id                                            object
type                                          object
number                                         int64
status                                        object
target_account_type                           object
payment_terms                                 object
include_balance_forward                         bool
enable_online_payment                           bool
include_statement_of_accounts                   bool
invoiced_at                                   object
due_at                           datetime64[ns, UTC]
voided_at                                     object
contactable_address                           object
contactable_name                              object
note                                          object
amount_cents                                   int64
amount_paid_cents                              int64
outstanding_amount_cents                       int64
payment_link                                  

## Load the formatted data frame into a BQ table

In [16]:
import pandas_gbq

project_id = 'www-prod-389819'
table_id = 'ohlaw.staging_lm_invoices'

# Load the DataFrame to BigQuery, replacing existing data
pandas_gbq.to_gbq(
    df,
    table_id,
    project_id=project_id,
    if_exists='replace'
)

print(f"DataFrame loaded to {table_id} in project {project_id} successfully, replacing existing data.")

100%|██████████| 1/1 [00:00<00:00, 8081.51it/s]

DataFrame loaded to ohlaw.staging_lm_invoices in project www-prod-389819 successfully, replacing existing data.
